In [1]:
# traing model
# import libary
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LeakyReLU, Conv2DTranspose, Activation, Reshape
from keras.layers import Conv2D, MaxPooling2D,UpSampling2D,ZeroPadding2D
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.backend.tensorflow_backend import set_session

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))
# 設定 Keras 使用的 Session

Using TensorFlow backend.


In [2]:
def getData(i, pack):
    print(i)
    path = '../../celebB/'
    data = np.zeros((pack,10000,64,64,3))
    for i in range(0,pack):
        originImageFile = 'celebA'+str(10000*(i+1))+'.npy'
        data[i] = np.load(path+originImageFile)
        print("A")
    data = np.reshape(data,(-1,64,64,3))
    print(data.shape)
    return data

In [3]:
def Discriminator():
    
    model = Sequential()
    depth = 64
    alpha=0.2
    # In: 28 x 28 x 1, depth = 1
    # Out: 14 x 14 x 1, depth=64
    input_shape = (64, 64, 3)
    model.add(Conv2D(depth*1, 5, strides=2, input_shape=input_shape,padding='same'))
    model.add(LeakyReLU(alpha))

    model.add(Conv2D(depth*2, 5, strides=2, padding='same'))
    model.add(LeakyReLU(alpha))

    model.add(Conv2D(depth*4, 5, strides=2, padding='same'))
    model.add(LeakyReLU(alpha))

    model.add(Conv2D(depth*8, 5, strides=2, padding='same'))
    model.add(LeakyReLU(alpha))

    # Out: 1-dim probability
    model.add(Flatten())
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

In [4]:
def Generator():
    model = Sequential()
    depth = 512
    dim = 4
    # In: 100
    # Out: dim x dim x depth
    model.add(Dense(dim*dim*depth, input_dim=100))
    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
    model.add(Reshape((dim, dim, depth)))

    # In: dim x dim x depth
    # Out: 2*dim x 2*dim x depth/2
    model.add(Conv2DTranspose(int(depth/2), 5, strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
    model.add(Activation('relu'))

    model.add(Conv2DTranspose(int(depth/4), 5, strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
    model.add(Activation('relu'))

    model.add(Conv2DTranspose(int(depth/8), 5, strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
    model.add(Activation('relu'))

    # Out: 28 x 28 x 1 grayscale image [0.0,1.0] per pix
    model.add(Conv2DTranspose(3, 5, strides=2, padding='same'))
    model.add(Activation('tanh'))
    return model

In [5]:
discriminator = Discriminator()
generator = Generator()
discriminator.summary()
generator.summary()

discriminatorModel = Sequential()
discriminator.trainable = True
discriminatorModel.add(discriminator)
discriminatorModel.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.0002),metrics=['accuracy'])

adversarialModel = Sequential()
adversarialModel.add(generator)
discriminator.trainable = False
adversarialModel.add(discriminator)
adversarialModel.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.0002),metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        4864      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       204928    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 256)         819456    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 4, 512)         3277312   
__________

In [6]:
trainSteps = 1000
batch_size = 64

print("training...")
for j in range(trainSteps):
    
    bigdata = getData(j%20, 1)
    bigdata = np.array_split(bigdata,10000/batch_size+1)
    for i in range(0,len(bigdata)):
    
        data = bigdata[i]
        
        noise = np.random.uniform(-1.0, 1.0, size=[len(data), 100])
        images_fake = generator.predict(noise)
        images_true = data
        x = np.concatenate((images_true, images_fake))
        y = np.ones([len(images_true) + len(images_fake), 1])
        y[len(images_true):, :] = 0
        discriminator.trainable = True
        d_loss = discriminatorModel.train_on_batch(x, y)
        discriminator.trainable = False
        a_loss = adversarialModel.train_on_batch(noise, np.ones([len(data), 1]))
        log = "%5d d:%f %f a:%f %f"%(j+1,d_loss[0],d_loss[1],a_loss[0],a_loss[1])
        print(log)
    print("save image {}".format(log))
    sys.stdout.write('\n')
    images_fake = generator.predict(noise)
    f,ax = plt.subplots(4,4)
    for ii in range(4):
        for jj in range(4):
            index=ii*4+jj
            ax[ii,jj].imshow(images_fake[index]/2+0.5)
    filename = "images/fakeFace%05d.png" % (j+1)
    plt.savefig(filename)

training...
0
A
(10000, 64, 64, 3)
    1 d:0.693194 0.265625 a:0.267772 1.000000
    1 d:0.434598 0.500000 a:0.000151 1.000000
    1 d:0.368551 0.500000 a:0.007059 1.000000
    1 d:0.341354 1.000000 a:0.004088 1.000000
    1 d:0.333426 1.000000 a:0.003147 1.000000
    1 d:0.313665 1.000000 a:0.002341 1.000000
    1 d:0.259647 1.000000 a:0.002034 1.000000
    1 d:0.149603 1.000000 a:0.000638 1.000000
    1 d:0.036196 1.000000 a:0.000156 1.000000
    1 d:0.006509 1.000000 a:0.000088 1.000000
    1 d:0.002108 1.000000 a:0.000062 1.000000
    1 d:0.000958 1.000000 a:0.000044 1.000000
    1 d:0.000761 1.000000 a:0.000024 1.000000
    1 d:0.000350 1.000000 a:0.000020 1.000000
    1 d:0.000118 1.000000 a:0.000019 1.000000
    1 d:0.000081 1.000000 a:0.000023 1.000000
    1 d:0.000056 1.000000 a:0.000016 1.000000
    1 d:0.000067 1.000000 a:0.000016 1.000000
    1 d:0.000023 1.000000 a:0.000014 1.000000
    1 d:0.000029 1.000000 a:0.000011 1.000000
    1 d:0.000013 1.000000 a:0.000011 1.000000

KeyboardInterrupt: 

In [ ]:
generator.save("fakeFaceModel.h5")